# Konvolucija

Konvolucija se može posmatrati kao kompozicija dveju funkcija od kojih jedna predstavlja signal, a druga, obično jednostavnija, filter koji se nad tim signalom primenjuje. U kontekstu obrade slika, govori se o konvoluciji matrica. Veća matrica dimenzija $m \times n$ predstavlja signal, a filter je obično znatno manja matrica, na primer, dimenzija $3 \times 3$. Počev od gornjeg levog ugla, filter polako klizi po matrici pokrivajući uvek deo veličine $3 \times 3$. Kada stigne do kraja reda, filter se spusta red niže, a postupak se ponavlja sve dok se ne dostigne donji desni ugao (videti animaciju ispod). Rezultat primene konvolucije je, takođe, matrica čije vrednosti zavise od prirode filtera i operacije koja je vršena. Na primer, često se vrši množenje odgovarajućih elemenata, a zatim se te vrednosti sabiraju ili usrednjavaju.

<img src="assets/konvolucija.png" width="400px">

<img src="assets/numerical_padding_strides.gif" width="400px">

Pomeraj (engl. stride) predstavlja veličinu pomeraja filter matrice duž x i duž y ose. U gornjem primeru to je vrednost 1 za obe ose. Proširenje (engl. padding) predstavlja dodavanje vrsta i kolona oko polazne matrice kako bi se ispunila očekivanja u pogledu dimenzija rezultujuće matrice. U zavisnosti od efekta koji treba da se postigne dodate vrste i kolone mogu sadržati određene vrednosti ili predstavljati refleksiju prvih nekoliko vrsta ili kolona polazne matrice. 

Primenom filtera se obično želi realizovati unapred definisana operacija, pa tako postoje i filteri specijalno dizajnirani za različite primene. Na primer, postoji filter za detekciju ivica, filter za izoštravanje slike, filter za zamućivanje slike i slično. Ovaj princip nalazi primenu kod konvolutivnih neuronskih mreža gde se koristi na nivou konvolutivnih slojeva za detekciju odgovarajućih karakteristika. Ovaj tip mreža s vremenom dobija na popularnosti i vrlo intenzivno se koristi u raznim zadacima, npr. u prepoznavanju objekata na slikama. Jedan zanimljiv pregled o konvolutivnim mrežama možete pronaći [ovde](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from PIL import Image
from PIL import ImageFilter

In [3]:
img = Image.open('data/art.png').convert('L')

Za specifične efekte se dizajniraju posebni filteri. Sledeći primer ilustruje filtere kojima se može detektovati prisustvo ivica.

Konstruisaćemo tri slike predstavljene odgovarajućim *numpy* matricama dimenzija $2 \times 2$. Svojstva koje slike treba da zadovolje su:

* Prva slika treba da ima sve elemente jednake, odnosno, traba da bude jednobojna. 
* Druga slika treba da ima jednake vrednosti u prvom i jednake vrednosti u drugom redu, odnosno treba da ima dve boje i horizontalnu ivicu.
* Treća slika treba da ima jednake vrednosti u prvoj i jednake vrednosti u drugoj koloni, odnosno treba da ima dve boje i vertikalnu ivicu.

`Gausov filter` (kaže se i kernel) se koristi za ujednačavanje (engl. smoothing) sadržaja slike. Jedna od matrica kojom se opisuje je $
\frac{1}{256}
\begin{bmatrix}
1 & 4 & 6 & 4 & 1 \\
4 & 16 & 24 & 16 & 4 \\
6 & 24 & 36 & 24 & 6 \\
4 & 16 & 24 & 16 & 4 \\
1 & 4 & 6 & 4 & 1 \\
\end{bmatrix}
$. 

In [4]:
gaussian_filter = (1/256) * np.array([
    [1, 4, 6, 4, 1],
    [4, 16, 24, 16, 4],
    [6, 24, 36, 24, 6],
    [4, 16, 24, 16, 4],
    [1, 4, 6, 4, 1]
])

Sličnu funkciju ima i `filter usrednjavanja` opisan matricom $ \frac{1}{16}
\begin{bmatrix}
1 & 2 & 1 \\
2 & 4 & 2 \\
1 & 2 & 1 \\
\end{bmatrix}
$.

In [5]:
averaging_filter = (1/16) * np.array([
    [1, 2, 1], 
    [2, 4, 2], 
    [1, 2, 1]
])

Sada možemo predefinisati funkciju `apply_filter` tako da radi nad proizvoljnom slikom i proizvoljnim filterom. Implementacija funkcije će podržavati proširenja početne slike nulama kako bi se dobila rezultujuća slika istih dimenzija. Prilikom primene filtera koristiće se pomeraj veličine 1. 

In [6]:
def apply_filter(image, filter):
    pass

Možemo demonstrirati i transformaciju slike Sobelovim filterom koji se koristi za detekciju ivica na slikama. On je određen dvema matricama $ G_{x} = 
\begin{bmatrix}
-1 & 0 & 1 \\
-2 & 0 & 2 \\
-1 & 0 & 1 \\
\end{bmatrix}
$ i $ G_{y} = 
\begin{bmatrix}
-1 & -2 & -1 \\
0 & 0 & 0 \\
1 & 2 & 1 \\
\end{bmatrix}
$ koje detektuju promene intenziteta na slikama, redom, duž x i y ose. Finalna transformacija je određena sa $\sqrt{I_x^2 + I_y^2}$ gde $I_x$ i $I_y$ predstavljaju transformacije slike dobijene duž osa.

In [7]:
sobel_x_direction_filter = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
])

In [8]:
sobel_y_direction_filter = np.array([
    [-1, -2, -1],
    [0, 0, 0], 
    [1, 2, 1]
])

In [28]:
sobel_x_transformed = apply_filter(image, sobel_x_direction_filter)
sobel_y_transformed = apply_filter(image, sobel_y_direction_filter)

In [29]:
image_with_sobel_filter = np.sqrt(sobel_x_transformed**2 + sobel_y_transformed**2).astype(int)

In [ ]:
plt.figure(figsize=(7, 7))
plt.axis('off')
plt.imshow(image_with_sobel_filter, cmap='gray')
plt.show()

Bibliotečka podrška za rad sa konvolucijom nad slikama dolazi kroz paket `signal` i njenu funkciju `convolve2d`. Funkcija osim slike i filtera očekuje parametar `mode` kojim se kontroliše proširenje i parametar `boundary` kojim se određuje priroda proširenja. 

In [9]:
from scipy import signal